In [ ]:
# train_densenet_soil.ipynb

import os
import numpy as np
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms

# --- Config ---
IMAGE_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 10
IMAGE_DIR = '/kaggle/input/soil-classification/soil_classification-2025/train/'
CSV_PATH = '/kaggle/input/soil-classification/soil_classification-2025/train_labels.csv'
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- Load labels ---
df = pd.read_csv(CSV_PATH)
df['image_path'] = df['image_id'].apply(lambda x: os.path.join(IMAGE_DIR, x))

# Encode soil_type labels
le = LabelEncoder()
df['label'] = le.fit_transform(df['soil_type'])
num_classes = len(le.classes_)

# --- Dataset & Transforms ---
transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

class SoilDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = self.df.loc[idx, 'image_path']
        label = self.df.loc[idx, 'label']
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label

# --- Split train and val ---
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])

train_dataset = SoilDataset(train_df, transform=transform)
val_dataset = SoilDataset(val_df, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

# --- Model ---
base_model = models.densenet121(pretrained=True)
for param in base_model.parameters():
    param.requires_grad = False

base_model.classifier = nn.Sequential(
    nn.Linear(base_model.classifier.in_features, 256),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(256, num_classes)
)
base_model = base_model.to(DEVICE)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(base_model.classifier.parameters())

def calculate_min_f1(model, dataloader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for images, labels in dataloader:
            images = images.to(DEVICE)
            outputs = model(images)
            preds = torch.argmax(outputs, dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels.numpy())
    f1_scores = f1_score(all_labels, all_preds, average=None)
    return np.min(f1_scores), f1_scores

best_min_f1 = -1
best_model_path = 'densenet121_best_minf1.pth'

for epoch in range(EPOCHS):
    base_model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = base_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    min_f1, _ = calculate_min_f1(base_model, val_loader)
    print(f"Epoch {epoch+1}/{EPOCHS} - Loss: {epoch_loss:.4f} - Min F1: {min_f1:.4f}")

    if min_f1 > best_min_f1:
        print(f"→ New best min F1 score! Saving model to {best_model_path}")
        best_min_f1 = min_f1
        torch.save(base_model.state_dict(), best_model_path)

# Save label classes
np.save("label_classes.npy", le.classes_)

print("Training complete.")
print(f"Best model saved to {best_model_path}")

